# OMERO Image Annotation Workflow with micro-SAM
This notebook guides you through annotating images in OMERO using the micro-SAM annotation plugin. Each step is explained clearly.

## 1. Setup and Installation
Install and import the required libraries. Make sure you have access to OMERO and the omero-annotate-ai package.

In [1]:
# Import the main package and check dependencies
import omero_annotate_ai
from omero_annotate_ai import (
    create_omero_connection_widget,
    create_workflow_widget,
    create_pipeline
 )

import os
from pathlib import Path

print(f"omero-annotate-ai version: {omero_annotate_ai.__version__}")
try:
    import ezomero
    print("OMERO functionality: Available")
except ImportError:
    print("OMERO functionality: Not available. Install with: pip install -e .[omero]")

omero-annotate-ai version: 0.1.2
OMERO functionality: Available


## 2. Connect to OMERO
Use the widget below to connect to your OMERO server. Fill in your server details and test the connection before proceeding.

In [3]:
# Create and display the OMERO connection widget
print("OMERO Connection Setup")
conn_widget = create_omero_connection_widget()
conn_widget.display()
print("\nAfter connecting, run the next cell to continue.")

OMERO Connection Setup
Loaded configuration from connection history: root@lcows1



After connecting, run the next cell to continue.


In [4]:
# Get the OMERO connection
conn = conn_widget.get_connection()

if conn is None:
    raise ConnectionError("No OMERO connection established. Please use the widget above to connect.")

print("OMERO connection established.")

OMERO connection established.


## 3. Configure the Annotation Workflow
Set up your annotation workflow using the widget below. Select your working directory, OMERO container, and configure annotation parameters.

In [ ]:
# Create and display the workflow widget
print("Annotation Workflow Setup")
workflow_widget = create_workflow_widget(connection=conn)
workflow_widget.display()
print("\nAfter completing the workflow, run the next cell.")

Annotation Workflow Setup



After completing the workflow, run the next cell.


## 4. Review and Run the Annotation Pipeline
Preview your configuration, validate your OMERO selection, and run the annotation pipeline.

#### 4.1. Configuration summary

In [8]:
# Get configuration from workflow widget
config = workflow_widget.get_config()

print("Configuration Summary:")
print(f"Container: {config.omero.container_type} (ID: {config.omero.container_id})")
print(f"Training Set: {config.name}")
print(f"Channel: {config.spatial_coverage.channels}")
print(f"Output: {config.output.output_directory}")
print(f"Resume from Table: {config.workflow.resume_from_table}")
print(f"Read-only Mode: {config.workflow.read_only_mode}")

if config.spatial_coverage.use_patches:
    print(f"Patches: {config.spatial_coverage.patches_per_image} per image ({config.spatial_coverage.patch_size[0]}×{config.spatial_coverage.patch_size[1]})")

if config.spatial_coverage.three_d:
    print("3D processing: Enabled")

Configuration Summary:
Container: screen (ID: 13)
Training Set: testscreen_20260218_183216
Channel: [0]
Output: /var/home/maartenpaul/omero_annotate_ai/26omero_annotations
Resume from Table: False
Read-only Mode: False
Patches: 1 per image (200×200)


#### 4.2. Convert configuration to an annotation pipeline

In [9]:
# Create pipeline and preview what will be processed
pipeline = create_pipeline(config, conn)

container_type = config.omero.container_type
container_id = config.omero.container_id

print(f"Validating {container_type} with ID {container_id}...")
container = conn.getObject(container_type.capitalize(), container_id)
if container is None:
    raise ValueError(f"{container_type.capitalize()} with ID {container_id} not found")
print(f"Found {container_type}: {container.getName()}")

Validating screen with ID 13...
Found screen: TestScreen


#### 4.3. Run the annotation pipeline on the files
After loading of the files the napari will open for you to annotate the images

In [ ]:
# Run the complete annotation pipeline using the updated full workflow function
print("Starting annotation pipeline...")
print(f"Using micro-SAM model: {config.ai_model.pretrained_from}")

if config.workflow.batch_size == 0:
    print("Processing: All images in one batch")
else:
    print(f"Processing: Batches of {config.workflow.batch_size} images")

print("Napari will open for interactive annotation.")

try:
    # Run the complete micro-SAM workflow (setup, schema, table, annotation)
    table_id, updated_config = pipeline.run_full_micro_sam_workflow()
    print("Annotation pipeline completed successfully.")
    print(f"Tracking table ID: {table_id}")
    if updated_config.workflow.read_only_mode:
        print(f"Annotations saved locally to: {updated_config.output.output_directory}")
    else:
        print("Annotations uploaded to OMERO")
    print(f"Total images processed: {len(updated_config.get_processed())}")
except Exception as e:
    print(f"Error during annotation pipeline: {e}")
    import traceback
    traceback.print_exc()

Starting annotation pipeline...
Using micro-SAM model: vit_b_lm
Processing: All images in one batch
Napari will open for interactive annotation.
Loading image IDs from screen 13
  Found 192 image IDs from container 13
Found 192 image IDs
Loaded 5 images (by ID)
Defining annotation schema...
Image smaller than patch size, using full image
Image smaller than patch size, using full image
Image smaller than patch size, using full image
Image smaller than patch size, using full image
Image smaller than patch size, using full image
Schema defined: 5 annotations
Creating OMERO tracking table...
Created/replaced tracking table 'testscreen_20260218_183216' with 5 units
   Primary container: screen 13
   Table ID: 2101
Created OMERO table with ID: 2101
Running micro-SAM annotation on 5 unprocessed annotations

Processing batch 1/1
   Batch size: 5
   Config state before: 0/5 processed
   Running annotation...
Loaded 2D data shape: (50, 50)
Image data range: 0 - 248, dtype: uint8
Loaded 2D data s

#### Save the configuration file to the local folder

In [ ]:
# Export configuration for future use
config_filename = f"annotation_config_{config.name}.yaml"
config_path = Path(config.output.output_directory) / config_filename

try:
    config.save_yaml(config_path)
    print(f"Configuration saved to: {config_path}")
except Exception as e:
    print("Could not save to output folder, saving to current directory")
    config.save_yaml(config_filename)
    print(f"Configuration saved to: {config_filename}")

print("To reuse this configuration:")
print("from omero_annotate_ai import load_config")
print(f"config = load_config('{config_filename}')")

## 5. Cleanup (Recommended)
Close the OMERO connection when you are finished. This is good practice, especially when working with shared servers.

In [ ]:
# Close OMERO connection (optional, but recommended)
if 'conn' in locals() and conn is not None:
    conn.close()
    print("OMERO connection closed.")